In [1]:
import pandas as pd
import glob
import os


# Import data
filename = 'outputs/results/combined_enterprise_output.csv'

df = pd.read_csv(filename)


In [5]:
import datetime
from cve_matching import translate_vendor, fetch_CVEs, fetch_CVEs_with_split

# Filter vendors
excluded_vendors_file = "config/excluded_vendors.txt"
with open(excluded_vendors_file, 'r', encoding='utf-8') as file:
    excluded_vendors = {line.strip() for line in file if line.strip()}

size_before = df.shape[0]
print(excluded_vendors)
df = df[~df["Vendor"].isin(excluded_vendors)]
print("Dropped " + str(size_before - df.shape[0]) + " by vendor exclusion.\n" + str(df.shape[0]) + " entries left.")
print(df.columns)
test_vendor = df["Vendor"][1]
test_time = df["Engine Time"][1]

end_date = datetime.datetime.now()

vendor_translated = translate_vendor(test_vendor)
print(vendor_translated)
cve_results = fetch_CVEs_with_split(vendor_translated, test_time, end_date=end_date)
#cve_results = fetch_CVEs(vendor_translated, test_time, end_date=end_date)

print(test_time)
print(cve_results)


{'ciscoSystems', 'HUAWEI Technology Co.,Ltd', 'net-snmp', 'Juniper Networks, Inc.'}
Dropped 0 by vendor exclusion.
8469 entries left.
Index(['IP', 'Enterprise Code', 'MAC', 'Engine Time', 'Engine Boots',
       'Vendor'],
      dtype='object')
no_match

DataFrame:
Empty DataFrame
Columns: []
Index: []

DataFrame:
Empty DataFrame
Columns: []
Index: []

DataFrame:
Empty DataFrame
Columns: []
Index: []

DataFrame:
Empty DataFrame
Columns: []
Index: []

DataFrame:
Empty DataFrame
Columns: []
Index: []

DataFrame:
Empty DataFrame
Columns: []
Index: []

DataFrame:
Empty DataFrame
Columns: []
Index: []

DataFrame:
Empty DataFrame
Columns: []
Index: []

DataFrame:
Empty DataFrame
Columns: []
Index: []

DataFrame:
Empty DataFrame
Columns: []
Index: []
49561381.0
Empty DataFrame
Columns: []
Index: []
